#Import needed modules

In [0]:
import math
import re
from collections import defaultdict, Counter
from nltk import ngrams
import pandas as pd
import glob as gb
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [0]:
with open('corpus.sent.en.train', 'r', encoding='utf8') as f:
    lines = f.readlines()
    

#Tokenizer

In [0]:
def tokenize(text):
    "List all the word tokens (consecutive letters) in a text. Normalize to lowercase."
    return re.findall('[a-z]+', text.lower())

#word-gram method

In [0]:
def word_ngrams(sent, n):
    """Givne a sent as str return n-grams as a list of tuple"""
    
    # EXAMPLES 
    # > word_ngrams('hello world', 1)
    # [('hello',), ('world',)]
    # > word_ngrams('hello world', 2)
    # [('<s>', 'hello'), ('hello', 'world'), ('world', '</s>')]

    # YOUR CODE HERE
    tp=()
    listtp= []
    token = tokenize(sent)
    #if it is not 1-gram append <s> to the begining and </s> at the end
    if(n!=1):
        for x in range(n-1):
            token.insert(0,"<s>")
        token.append("</s>")
    
    #generate n-grams using the zip function in python
    ngrams = zip(*[token[i:] for i in range(n)])
    
    #loop through the n-grams generated and create list of tuples
    for ngram in ngrams:
        listtp.append(ngram)
        
    return listtp

In [0]:
#We declare this method to generate the n-grams assisting the 
def getGram(lines, n):
    bigramst=[]
    if type(lines) == list:
        for sentences in lines:
            bg=word_ngrams(sentences, n)
            bigramst+=bg
    else:
        bigramst = word_ngrams(lines, n)
    return bigramst

In [0]:
# ADD YOUR CODE TO COLLECT COUTNS AND CONSTRUCT VOCAB
one_gram = getGram(lines, 1)
two_gram = getGram(lines, 2)
ugCount=Counter(one_gram)
bgCount=Counter(two_gram)

In [0]:
#     total_counts = sum(ngram_counts.values())
onevocab=vocab = list(ugCount.keys())

biVocab = []
for key in ugCount.keys():
    biVocab.extend(tokenize(str(key)))

biVocab.append('<s>')
biVocab.append('</s>')



The functions for the probabilties start here

In [0]:
#function N for the number of count of word w
def N(word):
  N=ugCount[(word,)]
  return N
  

In [0]:
print("N('york') = "+str(N('york')))
print("N('matter') = "+str(N('matter')))

N('york') = 2374
N('matter') = 2367


In [0]:
#create a count dictionary for all words in the bigram when they are the centre word
histCount = Counter()
centreWordCount = Counter()
prefixWordCount = Counter()
for key, value in bgCount.items():
  histCount[key[:-1]] += value
for key, value in bgCount.items():
  centreWordCount[key[1]] += value
for key, value in bgCount.items():
  prefixWordCount[key[1]] += 1

In [0]:
#Number of count N1+
def N1plus(word):
  return prefixWordCount[word]
  

In [0]:
print("N1+(.'york') = "+str(N1plus('york')))
print("N1+(.'matter') = "+str(N1plus('matter')))

N1+(.'york') = 9
N1+(.'matter') = 270


In [0]:
#is maximum likelihood estimation of the unigram probability (no smoothing)
def logProbML(word):
  probability=ugCount[(word,)]/len(one_gram)
  return -1*math.log(probability, 2)

In [0]:
print("-logPml('york') = "+str(logProbML('york')))
print("-logPml('matter') = "+str(logProbML('matter')))

-logPml('york') = 12.258374283571047
-logPml('matter') = 12.262634512478337


In [0]:
#Lidstone-smoothed unigram probability
def logProbLds(word,alpha=1):
  probability=(alpha + ugCount[(word,)])/(alpha*len(vocab) + len(one_gram))
  return -1*math.log(probability, 2)

In [0]:
print("-logPlids('york') = "+str(logProbLds('york')))
print("-logPlids('matter') = "+str(logProbLds('matter')))

-logPlids('york') = 12.268770360977008
-logPlids('matter') = 12.27302879345373


In [0]:
#the Kneser-Ney unigram probability defined in Equation (5).
def logProbKn(word):
  probability=N1plus(word)/len(bgCount)
  return -1*math.log(probability, 2)

In [0]:
print("-logPkn('york') = "+str(logProbKn('york')))
print("-logPkn('matter') = "+str(logProbKn('matter')))

-logPkn('york') = 17.964084485236466
-logPkn('matter') = 13.057193889627946
